In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

from bs4 import BeautifulSoup as bs

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from selenium.webdriver.common.keys import Keys
import pyperclip

import pandas as pd

### 네이버 웹툰 페이지 열기

In [ ]:
# 네이버 웹툰 페이지 열기
url = "https://comic.naver.com/webtoon"
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)

# 페이지 로딩을 위한 시간
time.sleep(3)

### 청소년 관람불가 웹툰 정보도 가져오기 위한 로그인 과정

In [ ]:
# 네이버 웹툰 로그인
user_id = 'naver_id'
user_pw = 'naver_password'

login_btn = driver.find_element_by_class_name('gnb_btn_login')
login_btn.click()

# ID
login_id = driver.find_element_by_id('id')
login_id.click()
pyperclip.copy(user_id)
login_id.send_keys(Keys.COMMAND, 'v')
time.sleep(2)

# Password
login_pw = driver.find_element_by_id('pw')
login_pw.click()
pyperclip.copy(user_pw)
login_pw.send_keys(Keys.COMMAND, 'v')
time.sleep(2)

# 5. 로그인 버튼 클릭
driver.find_element_by_id('log.login').click()

### CSV 파일로 변환하기 위한 정기 연재 웹툰 리스트 선언

In [ ]:
# 정보들을 담을 리스트 정의하기
id_list = []
title_list = []
author_list = []
genre_list = []
story_list = []
webtoon_url_list = []
thumbnail_url_list = []

webtoon_id = 0

### 정기 연재 웹툰 정보 가져오기

In [ ]:
# 주 1회 연재중인 클릭할 수 있는 제목 리스트 가져오기
titles = driver.find_elements(By.CLASS_NAME, "Poster__link--sopnC")

In [ ]:
# 주 1회 연재 웹툰 개수만큼 반복하기
for i in range(len(titles)):
    print("\rprocess: " + str(i + 1) + " / " + str(len(titles)), end="")

    # 웹페이지가 로딩되기도 전에 코드가 실행되는 것을 방지하기 위한 기다림
    time.sleep(1)

    # 0번째 웹툰, 즉 월요일 첫번재 웹툰부터 클릭해서 해당 페이지로 이동하기
    titles = driver.find_elements(By.CLASS_NAME, "Poster__link--sopnC")
    
    # 클릭하려는 요소로 스크롤하여 가시화시키기
    driver.execute_script("arguments[0].scrollIntoView();", titles[i])
    
    # 클릭하기
    titles[i].click()
    
    # 웹페이지가 완전히 로드될 때까지 기다리는 시간 설정 (예: 10초)
    wait = WebDriverWait(driver, 10)
    
    # 웹페이지가 완전히 로드될 때까지 기다림
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'EpisodeListInfo__title--mYLjC')))
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'Poster__image--d9XTI')))
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ContentMetaInfo__link--xTtO6')))
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'TagGroup__tag_group--uUJza')))
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'EpisodeListInfo__summary_wrap--ZWNW5')))

    # 이동한 페이지의 html 코드 가져오기
    html = driver.page_source
    soup = bs(html, 'html.parser')

    # 제목 정보 가져오기
    title = soup.find('h2', {'class': 'EpisodeListInfo__title--mYLjC'}).text
    
    # 만약 연재 요일이 2개 이상이라서 이미 저장했던 웹툰이라면 넘어가기
    if title in title_list:
        driver.back()
        continue
    
    # 썸네일 이미지 주소 가져오기
    # thumbnail_url = soup.find('img', {'class': 'Poster__image--d9XTI'})['src']
    thumbnail_url = soup.find('div', {'class': 'Poster__thumbnail_area--gviWY Poster__type193x250--Ge81Q'}).find('img')['src']
    
    # 저자 정보 가져오기
    author =  soup.find('a', {'class': 'ContentMetaInfo__link--xTtO6'}).text
    author = author.replace(' / ', ', ')
    
    # 카테고리 태그 정보 가져오기
    tag_group = soup.find('div', {'class': 'TagGroup__tag_group--uUJza'})
    tags = tag_group.find_all('a')
    tag_group = [tag.text.replace('#', '') for tag in tags]
    
    # 장르 정보 가져오기
    genre = tag_group[0]
    
    # 줄거리 정보 가져오기
    story = soup.find('div', {'class': 'EpisodeListInfo__summary_wrap--ZWNW5'}).find('p').text

    # 정보들을 리스트에 담기
    id_list.append(webtoon_id)
    title_list.append(title)
    thumbnail_url_list.append(thumbnail_url)
    webtoon_url_list.append(driver.current_url)
    author_list.append(author)
    genre_list.append(genre)
    story_list.append(story)
    
    # 뒤로 가기
    driver.back()
    webtoon_id += 1
    time.sleep(0.5)

### 정기 연재 웹툰 정보 CSV 파일로 저장

In [ ]:
total_data = pd.DataFrame()

total_data['id'] = id_list
total_data['title'] = title_list
total_data['author'] = author_list
total_data['genre'] = genre_list
total_data['story'] = story_list
total_data['thumbnail_url'] = thumbnail_url_list
total_data['webtoon_url'] = webtoon_url_list

# 따로 인덱스를 생성하지 않고 id를 인덱스로 정하기
total_data.set_index('id', inplace=True)

# CSV 파일로 저장하기
total_data.to_csv("daily.csv", encoding='utf-8-sig')

### CSV 파일로 변환하기 위한 정기 연재 웹툰 리스트 선언

In [ ]:
# 정보들을 담을 리스트 정의하기
id_list = []
title_list = []
author_list = []
genre_list = []
story_list = []
webtoon_url_list = []
thumbnail_url_list = []

webtoon_id = 0

### 비정기 연재 웹툰 정보 가져오기

In [ ]:
# 비정기 연재 페이지로 넘어가기
element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//a[contains(text(), '매일+')]"))
)

href_attribute = element.get_attribute('href')
driver.get(href_attribute)

In [ ]:
# 비정기 연재중인 클릭할 수 있는 제목 리스트 가져오기
titles = driver.find_elements(By.CLASS_NAME, "Poster__link--sopnC")

In [ ]:
# 비정기 연재 웹툰 개수만큼 반복하기
for i in range(len(titles)):
    print("\rprocess: " + str(i + 1) + " / " + str(len(titles)), end="")

    # 웹페이지가 로딩되기도 전에 코드가 실행되는 것을 방지하기 위한 기다림
    time.sleep(1)

    # 0번째 웹툰, 즉 월요일 첫번재 웹툰부터 클릭해서 해당 페이지로 이동하기
    titles = driver.find_elements(By.CLASS_NAME, "Poster__link--sopnC")
    
    # 클릭하려는 요소로 스크롤하여 가시화시키기
    driver.execute_script("arguments[0].scrollIntoView();", titles[i])
    
    # 클릭하기
    titles[i].click()
    
    # 웹페이지가 완전히 로드될 때까지 기다리는 시간 설정 (예: 10초)
    wait = WebDriverWait(driver, 10)
    
    # 웹페이지가 완전히 로드될 때까지 기다림
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'EpisodeListInfo__title--mYLjC')))
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'Poster__image--d9XTI')))
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ContentMetaInfo__link--xTtO6')))
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'TagGroup__tag_group--uUJza')))
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'EpisodeListInfo__summary_wrap--ZWNW5')))

    # 이동한 페이지의 html 코드 가져오기
    html = driver.page_source
    soup = bs(html, 'html.parser')

    # 제목 정보 가져오기
    title = soup.find('h2', {'class': 'EpisodeListInfo__title--mYLjC'}).text
    
    # 만약 연재 요일이 2개 이상이라서 이미 저장했던 웹툰이라면 넘어가기
    if title in title_list:
        driver.back()
        continue
    
    # 썸네일 이미지 주소 가져오기
    # thumbnail_url = soup.find('img', {'class': 'Poster__image--d9XTI'})['src']
    thumbnail_url = soup.find('div', {'class': 'Poster__thumbnail_area--gviWY Poster__type193x250--Ge81Q'}).find('img')['src']
    
    # 저자 정보 가져오기
    author =  soup.find('a', {'class': 'ContentMetaInfo__link--xTtO6'}).text
    author = author.replace(' / ', ', ')
    
    # 카테고리 태그 정보 가져오기
    tag_group = soup.find('div', {'class': 'TagGroup__tag_group--uUJza'})
    tags = tag_group.find_all('a')
    tag_group = [tag.text.replace('#', '') for tag in tags]
    
    # 장르 정보 가져오기
    genre = tag_group[0]
    
    # 줄거리 정보 가져오기
    story = soup.find('div', {'class': 'EpisodeListInfo__summary_wrap--ZWNW5'}).find('p').text

    # 정보들을 리스트에 담기
    id_list.append(webtoon_id)
    title_list.append(title)
    thumbnail_url_list.append(thumbnail_url)
    webtoon_url_list.append(driver.current_url)
    author_list.append(author)
    genre_list.append(genre)
    story_list.append(story)
    
    # 뒤로 가기
    driver.back()
    webtoon_id += 1
    time.sleep(0.5)

###  비정기 연재 웹툰 정보 CSV 파일로 저장

In [ ]:
total_data = pd.DataFrame()

total_data['id'] = id_list
total_data['title'] = title_list
total_data['author'] = author_list
total_data['genre'] = genre_list
total_data['story'] = story_list
total_data['thumbnail_url'] = thumbnail_url_list
total_data['webtoon_url'] = webtoon_url_list

# 따로 인덱스를 생성하지 않고 id를 인덱스로 정하기
total_data.set_index('id', inplace=True)

# CSV 파일로 저장하기
total_data.to_csv("dailyPlus.csv", encoding='utf-8-sig')